In [ ]:
import requests  
import json  
  
class SearchResult:  
    def __init__(self, data):  
        self.aggregations = data.get('aggregations', {})  
        self.total = data.get('total', 0)  
        self.pages = data.get('pages', 0)  
        self.currentPage = data.get('currentPage', 0)  
        self.currentPer = data.get('currentPer', 0)  
        self.results = data.get('results', [])  
      
    def to_dict(self):  
        return {  
            'aggregations': self.aggregations,  
            'total': self.total,  
            'pages': self.pages,  
            'currentPage': self.currentPage,  
            'currentPer': self.currentPer,  
            'results': [result for result in self.results]  
        }  
      
    def to_json(self):  
        return json.dumps(self.to_dict(), indent=4)  

In [1]:
import requests  
import json  
  
def search_etfs(query=''):  
    # Set default values for other parameters inside the function  
    per = 25  
    page = 1  
    filter = {}  
    sort = {'shareClassVolume': 'desc'}  
      
    url = 'https://search.finanzfluss.de/graphql'  
    graphql_query = """  
        fragment CoreSearchFields on SearchResult2 {  
            aggregations  
            total  
            unfilteredTotal  
            pages  
            currentPage  
            currentPer  
            __typename  
        }  
  
        fragment OverviewTableFields on ETF {  
            releaseDate  
            fundVolume  
            shareClassVolume  
            totalExpenseRatio  
            isDistributing  
            replicationMethod  
            currency  
            preferredPerformanceIdId  
            __typename  
        }  
  
        query InformerSearchQuery($query: String, $per: Int, $page: Int, $filter: JSON, $sort: JSON) {  
            search(q: $query, per: $per, page: $page, filter: $filter, sort: $sort) {  
                ...CoreSearchFields  
                results {  
                    isin  
                    name  
                    displayName  
                    preferredPerformanceIdId  
                    ...OverviewTableFields  
                    __typename  
                }  
                __typename  
            }  
        }  
    """  
  
    variables = {  
        'query': query,  
        'per': per,  
        'page': page,  
        'filter': filter,  
        'sort': sort  
    }  
  
    headers = {  
        'Content-Type': 'application/json; charset=utf-8'  
    }  
  
    response = requests.post(url, headers=headers, json={  
        'operationName': 'InformerSearchQuery',  
        'query': graphql_query,  
        'variables': variables  
    })  
  
    if response.status_code == 200:  
        data = response.json()  
        search_data = data['data']['search']  
  
        search_results = {  
            'aggregations': search_data.get('aggregations', {}),  
            'total': search_data.get('total', 0),  
            'pages': search_data.get('pages', 0),  
            'currentPage': search_data.get('currentPage', 0),  
            'currentPer': search_data.get('currentPer', 0),  
            'results': [result for result in search_data.get('results', [])]  
        }  
        return json.dumps(search_results, indent=4)  
    else:  
        response.raise_for_status()


In [2]:
try:  
    search_result = search_etfs(query='vwrl')  
    print(search_result)  
except requests.RequestException as e:  
    print(f"An error occurred: {e}")  


{
    "aggregations": {
        "asset_allocation": {
            "all": 0
        },
        "base_index": {
            "prioritized": {
                "FTSE All-World": 1
            },
            "not_prioritized": {}
        },
        "bond": {
            "bond_type": {
                "aggregate": 0,
                "convertibles": 0,
                "corporate_bonds": 0,
                "covered_bonds": 0,
                "government_bonds": 0
            },
            "bond_rating": {
                "AAA": 0,
                "investment_grade": 0,
                "mixed": 0,
                "none": 0,
                "sub_investment_grade": 0
            },
            "bond_maturity": {
                "0-1": 0,
                "0-10": 0,
                "0-3": 0,
                "0-5": 0,
                "1-10": 0,
                "1-15": 0,
                "10+": 0,
                "3-7": 0,
                "5+": 0,
                "5-10": 0,
                "5-7": 0,


In [3]:
system_prompt = """
You are a financial AI assistant specialized in handling queries related to securities like stocks, ETFs, and bonds. Your role is to interpret and extract key identifiers such as names, ISINs (International Securities Identification Number), or WKNs (Wertpapierkennnummer) from user queries.
- If a query does not include a clear identifier for a security, you should attempt to infer the necessary information using relevant context from the query. If you are unable to confidently extract or infer the required identifier, prompt the user to provide this information explicitly.
- Once an identifier is extracted, utilize the `search_etfs` function to retrieve detailed information about Exchange-Traded Funds (ETFs) based on the extracted identifier, which can be the ETF's name, ticker symbol, or ISIN.
- Clearly inform the user when you are making an inference about the identifier to ensure transparency.
- Your responses should be strictly based on the information retrieved from the `search_etfs` function. If no relevant information is found or the query cannot be answered with the available data, respond with "I don't know." Do not fabricate or guess answers.
"""

tools= [  
        {  
            "type": "function",  
            "function": {  
                "name": "search_etfs",  
                "description": "Retrieve detailed information about Exchange-Traded Funds (ETFs) based on a search query.",  
                "parameters": {  
                    "type": "object",  
                    "properties": {  
                        "query": {  
                            "type": "string",  
                            "description": "The search query used to find ETFs, which can include the ETF's name, ticker symbol, or ISIN."  
                        }  
                    },  
                    "required": ["query"]  
                }  
            }  
        }  
    ] 

In [13]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv 
  
# Load the .env file  
load_dotenv()  
  
# Now you can safely call os.getenv to retrieve the environment variables  
api_key = os.getenv("AZURE_OPENAI_API_KEY")  
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")  
api_version = os.getenv("AZURE_OPENAI_API_VERSION")  
model_name = os.getenv("AZURE_OPENAI_MODEL_NAME")  
  
# Ensure the variables are set  
if not api_key or not azure_endpoint or not api_version or not model_name:
    print(api_key)  
    print(azure_endpoint)
    print(api_version)
    print(model_name)
    raise ValueError("error loading env variables from .env file.")  
  
client = AzureOpenAI(  
    api_key=api_key,  
    api_version=api_version,  
    azure_endpoint=azure_endpoint  
)  
    
assistant = client.beta.assistants.create(  
    instructions=system_prompt,  
    model=model_name,  # replace with model deployment name  
    tools=tools  
)

# Create a thread
thread = client.beta.threads.create()

In [14]:
def call_functions(client: AzureOpenAI, thread, run) -> None:
    print("Function Calling")
    required_actions = run.required_action.submit_tool_outputs.model_dump()
    print(required_actions)
    tool_outputs = []

    for action in required_actions["tool_calls"]:
        func_name = action["function"]["name"]
        arguments = json.loads(action["function"]["arguments"])

        if func_name == "search_etfs":
            output = search_etfs(arguments["query"])
            tool_outputs.append({"tool_call_id": action["id"], "output": output})
        else:
            raise ValueError(f"Unknown function: {func_name}")

    print("Submitting outputs back to the Assistant...")
    client.beta.threads.runs.submit_tool_outputs(thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs)

In [15]:
def format_messages(messages) -> None:
    message_list = []

    # Get all the messages till the last user message
    for message in messages:
        message_list.append(message)
        if message.role == "user":
            break

    # Reverse the messages to show the last user message first
    message_list.reverse()

    # Print the user or Assistant messages or images
    for message in message_list:
        for item in message.content:
            print(f"{message.role}:\n{item.text.value}\n")

In [16]:
import time

def process_message(content: str) -> None:
    # Add a user question to the thread
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=content # Replace this with your prompt
    )

    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
    )

    print("processing...")
    while True:
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        if run.status == "completed":
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            format_messages(messages)
            break
        if run.status == "failed":
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            format_messages(messages)
            # Handle failed
            break
        if run.status == "expired":
            # Handle expired
            break
        if run.status == "cancelled":
            # Handle cancelled
            break
        if run.status == "requires_action":
            call_functions(client, thread, run)
        else:
            time.sleep(5)

In [17]:
process_message("What is the inception date of Fund Vanguard FTSE All-World UCITS ETF Distributing?")


processing...
Function Calling
{'tool_calls': [{'id': 'call_AtLCr2UAF1cGF6KYYdik6wlT', 'function': {'arguments': '{"query":"Vanguard FTSE All-World UCITS ETF Distributing"}', 'name': 'search_etfs'}, 'type': 'function'}]}
Submitting outputs back to the Assistant...
user:
What is the inception date of Fund Vanguard FTSE All-World UCITS ETF Distributing?

assistant:
The inception date of the Vanguard FTSE All-World UCITS ETF Distributing is May 22, 2012.



In [18]:
from assistant import Assistant  
  
# Create an instance of the Assistant class  
assistant = Assistant()  
  
# Define a test message query  
test_query = "What is the inception date of Fund Vanguard FTSE All-World UCITS ETF Distributing?"  
  
# Process the message using the Assistant instance  
response = assistant.process_message(test_query)  
  
# Display the response  
print(response)  

Processing...
Function Calling
{'tool_calls': [{'id': 'call_KL6yyQVNOxwJSAfVJ7sBEIMe', 'function': {'arguments': '{"query":"Vanguard FTSE All-World UCITS ETF Distributing"}', 'name': 'search_etfs'}, 'type': 'function'}]}
Submitting outputs back to the Assistant...
user:
What is the inception date of Fund Vanguard FTSE All-World UCITS ETF Distributing?

assistant:
The inception date of the Vanguard FTSE All-World UCITS ETF Distributing is May 22, 2012.

None
